In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
from pandas.plotting import parallel_coordinates
from sklearn import decomposition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
import seaborn as sn
from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA

In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import balanced_accuracy_score
    
def metric(p,t):
    mse = mean_squared_error(t,p)
    
    r = r2_score( t , p ,  multioutput='variance_weighted')
    
    return mse,r

In [2]:
#dataread

OP_df = pd.read_csv('OptionData12110.csv')
OP_df=OP_df.dropna()
OP_df['Pdiff'] =  OP_df['Strike_price'] - OP_df['Stock_price'] 
COP_df = OP_df. loc[OP_df['Option_type'] == 1]
POP_df = OP_df.loc[OP_df['Option_type'] == 0]
OP_df.head(1)

#OP_df['Pdiff'] = OP_df.apply(lambda x: int(x['Stock_price']) - int(x['Stock_price']), axis=1)
def func(x):
    if x > 0 :
        return 'OTM'
    elif x < 0 :
        return 'ITM'
    else:
        return 'ATM'
    
def pfunc(x):
    if x > 0 :
        return 'PUT_ITM'
    elif x < 0 :
        return 'PUT_OTM'
    else:
        return 'PUT_ATM'

COP_df['Type'] = COP_df['Pdiff'].apply(func)
POP_df['Type'] = POP_df['Pdiff'].apply(pfunc)

COP_df = COP_df.drop(['Pdiff'], axis = 1)
POP_df = POP_df.drop(['Pdiff'], axis = 1)

frames = [COP_df, POP_df]

df = pd.concat(frames)
df.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Option_type,Ask,Bid,Option_price,Stock_price,Strike_price,Volatility,Volume,Time_to_maturity,Implied_volatility,Type
0,0,1,0.2,0.0,0.09,79.54,45.0,0.286563,4,0.123288,0.763674,ITM


In [3]:
from sklearn.model_selection import train_test_split
df = df.drop(['Type'], axis = 1)
data = np.array(StandardScaler().fit_transform(df))
label = np.array(df.Implied_volatility)

X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.20, random_state=42)

In [5]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
np.shape(X_test)

(2422, 11, 1)

In [ ]:
#LSTM

In [4]:

from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import pandas as pd
import numpy as np

model = Sequential()
model.add(LSTM (64, return_sequences = True,input_shape = [11,1]))
model.add(Dropout(0.2))
model.add(LSTM (32))
model.add(Dropout(0.2))
model.add(Dense(1))


model.compile(loss='mse', optimizer='rmsprop' , metrics=['accuracy'] )

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11, 64)            16896     
                                                                 
 dropout (Dropout)           (None, 11, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = model.fit(X_train, Y_train, batch_size=32, epochs=500, validation_split = 0.1, verbose = 1)

In [ ]:
pred = model.predict(X_test)
mse_bst,r2_bst = metric(pred,Y_test)
print("SVM 1. MSE 2. R2 : " , mse_bst,r2_bst)

In [23]:
from sklearn.model_selection import train_test_split
#df = df.drop(['Type'], axis = 1)
data = np.array(StandardScaler().fit_transform(df))
label = np.array(df.Implied_volatility)

X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.20, random_state=42)

In [ ]:
#Transformer

In [43]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)

In [34]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
np.shape(X_test)

(2422, 11, 1)

In [44]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="mse",
    optimizer='rmsprop',
    metrics=['accuracy'],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    Y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    callbacks=callbacks,
)


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 11, 1)]      0           []                               
                                                                                                  
 layer_normalization_26 (LayerN  (None, 11, 1)       2           ['input_11[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_13 (Multi  (None, 11, 1)       7169        ['layer_normalization_26[0][0]', 
 HeadAttention)                                                   'layer_normalization_26[0][0]'] 
                                                                                            

In [46]:
pred = model.predict(X_test)
mse_bst,r2_bst = metric(pred,Y_test)
print("1. MSE 2. R2 : " , mse_bst,r2_bst)

1. MSE 2. R2 :  0.00013612616657376553 0.9961074374361188


In [42]:
np.shape(pred)

(2422, 2)

In [ ]:
#ResNet

In [6]:
from sklearn.model_selection import train_test_split
#df = df.drop(['Type'], axis = 1)
data = np.array(StandardScaler().fit_transform(df))
label = np.array(df.Implied_volatility)

X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.20, random_state=42)

In [7]:
X_train = X_train.reshape(9688*11)
b = np.ones(952, dtype = int)
X_train = np.append(X_train,b)
X_train = X_train.reshape(35,32,32,3)
np.shape(X_train)

(35, 32, 32, 3)

In [8]:
X_test = X_test.reshape(2422*11)
b = np.ones(1006, dtype = int)
X_test = np.append(X_test,b)
X_test = X_test.reshape(9,32,32,3)
np.shape(X_test)

(9, 32, 32, 3)

In [9]:
Y_train = np.ones(35, dtype = int)
np.shape(Y_train)
Y_test = np.ones(9, dtype = int)
np.shape(Y_test)

(9,)

In [12]:
import keras
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

resnet = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    input_tensor=None,
    input_shape=(32, 32, 3),
    pooling=None
)

for layer in resnet.layers[:-4]:                                                       
    layer.trainable = False
    
#model.summary()

model = Sequential()

model.add(resnet) 
model.add(Dense (32, activation='relu'))
#Final Dense Layer
model.add(Dropout(.2))
model.add(Dense(1, activation="linear"))  
                     
model.compile(optimizer='rmsprop', loss='mse', metrics='accuracy')
hist = model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_split = 0.1, verbose = 1)

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 2.7605 - accuracy: 0.9355 - val_loss: 26.6809 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 234ms/step - loss: 53.2519 - accuracy: 0.0000e+00 - val_loss: 0.5480 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 228ms/step - loss: 2.8233 - accuracy: 0.5161 - val_loss: 0.1884 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 221ms/step - loss: 2.6118 - accuracy: 0.4839 - val_loss: 0.1758 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 217ms/step - loss: 3.3134 - accuracy: 0.6129 - val_loss: 0.0945 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 261ms/step - loss: 1.9424 - accuracy: 0.8065 - val_loss: 0.1499 - val_accuracy: 0.7500
Epoch 7/10
1/1 [==============================] - 0s 264ms/step - loss: 0.5765 - accuracy: 0.6129 - val_loss: 0.1287 - val_accuracy: 0.7500
Epoch 8/10
1/

In [17]:
pred = model.predict(X_test)
mse_bst,r2_bst = metric(pred.reshape(len(pred),1),Y_test)
print("SVM 1. MSE 2. R2 : " , mse_bst,r2_bst)

SVM 1. MSE 2. R2 :  0.38172601948303075 0.0


In [16]:
np.shape(pred)

(9, 1, 1, 1)